## STEP 1. 형태소 분석기를 이용하여 품사에 따라 해당 단어를 추출하기

- 명사만 추출 `nouns.txt`
- 명사, 동사 추출 `nouns_verbs.txt`
- 명사, 동사, 형용사 추출 `nouns_verbs_adjectives.txt`

In [1]:
# 시간이 오래 소요되므로 파일로 저장
# word2vec 생성을 위한 토큰화
import os
from konlpy.tag import Okt
from tqdm import tqdm

okt = Okt()
input_file_path = os.getenv('HOME') + '/aiffel/weat/synopsis.txt'
output_file_path = './tokenized/nouns.txt'

'''
with open(input_file_path, 'r', encoding='utf-8') as rf:
    with open(output_file_path, 'w', encoding='utf-8') as wf:
        lines = rf.readlines()
        for line in tqdm(lines):
            words = okt.pos(line, stem=True, norm=True)
            res = [w[0] for w in words if w[1] == "Noun" or w[1] == "Verb"]
            wf.write(' '.join(res) + '\n')
'''

'\nwith open(input_file_path, \'r\', encoding=\'utf-8\') as rf:\n    with open(output_file_path, \'w\', encoding=\'utf-8\') as wf:\n        lines = rf.readlines()\n        for line in tqdm(lines):\n            words = okt.pos(line, stem=True, norm=True)\n            res = [w[0] for w in words if w[1] == "Noun" or w[1] == "Verb"]\n            wf.write(\' \'.join(res) + \'\n\')\n'

In [2]:
# 저장해놓은 토큰 파일 불러오기
tokenized=[]
with open(output_file_path, 'r', encoding='utf-8') as rf:
    for line in rf:
        # 각 줄을 읽어 공백을 기준으로 단어를 나누고 리스트로 변환
        words = line.strip().split()
        tokenized.append(words)

In [3]:
# 개수 세기
cnt=0
for words in tokenized:
    cnt+=len(words)
print(cnt)

1342179


- Noun 개수: 1342179
- Noun & Verb 개수: 1723183
- Noun & Verb & Adjective 개수: 1850315

## STEP 2. 추출된 결과로 embedding model 만들기

In [4]:
from gensim.models import Word2Vec

# tokenized에 담긴 데이터를 가지고 나만의 Word2Vec을 생성합니다. (Gensim 4.0 기준)
model = Word2Vec(tokenized, vector_size=100, window=5, min_count=3, sg=0)  
model.wv.most_similar(positive=['영화'])

[('작품', 0.8872395753860474),
 ('다큐멘터리', 0.8389464020729065),
 ('드라마', 0.812419056892395),
 ('코미디', 0.7906116843223572),
 ('영화로', 0.7843707799911499),
 ('형식', 0.780301034450531),
 ('주제', 0.7609272599220276),
 ('실화', 0.760643720626831),
 ('소재', 0.7489678859710693),
 ('스토리', 0.748929500579834)]

## STEP 3. target, attribute 단어 셋 만들기

In [5]:
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from konlpy.tag import Okt

# 토큰화 파일 저장 함수
def save_tokens(input_file_name, output_file_name):
    okt = Okt()
    result = []
    with open(os.getenv('HOME')+'/aiffel/weat/'+input_file_name, 'r') as fread: 
        with open('./tokenized/'+output_file_name, 'w', encoding='utf-8') as fwrite:
            print(input_file_name, '파일을 읽고 있습니다.')
            lines = fread.readlines()
            for line in tqdm(lines):
                words = okt.pos(line, stem=True, norm=True)
                res = [w[0] for w in words if w[1] == "Noun" or w[1] == "Verb"]
                fwrite.write(' '.join(res) + '\n')
# 토큰 파일 불러오는 함수              
def read_tokens(file_name):
    with open('./tokenized/' + file_name, 'r', encoding='utf-8') as fread:
        # 파일 전체를 읽어서 줄바꿈 문자를 제거하고 하나의 문자열로 반환
        tokenized = fread.read().replace('\n', ' ')
    
    return tokenized

명사만 추출할 때에는 `okt.nouns()`을 활용해보았지만 소요 시간 차이 크지 않았음

In [6]:
# 불용어 불러오기
with open('./stopwords.txt', 'r', encoding='utf-8') as fread:
    stopwords = [line.strip() for line in fread.readlines()]
    print(stopwords)

['하다', '되다', '그녀', '자신', '위해', '않다', '이다', '되어다', '싶다', '점점', '때문', '오다', '가다', '과연', '이제', '통해', '시작', '영화']


stopwords 구성 기준: 모든 데이터셋에서 자주 등장하고, 주제와 관련 없는 단어 선정

In [7]:
# LDA 결과 출력 함수
# 각 주제에 대한 단어 분포 출력
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic #{topic_idx}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

# 각 문서에서 가장 확률이 큰 주제와 그 확률 출력
def print_max_topic_for_documents(model, documents):
    topic_distribution = model.transform(X)
    max_topic_idxs=[]
    for i, doc in enumerate(documents):
        max_topic_idx = np.argmax(topic_distribution[i])
        max_prob = topic_distribution[i, max_topic_idx]
        print(f"Document #{i}:")
        print(f"Max Probability Topic: {max_topic_idx}, Probability: {max_prob:.4f}")
        print()
        max_topic_idxs.append(max_topic_idx)
    return max_topic_idxs
        
        
def get_topic_words(model, topic_idx, feature_names, n_top_words):
    topic_word_distribution = model.components_[topic_idx]
    top_word_indices = topic_word_distribution.argsort()[:-n_top_words-1:-1]
    top_words = [feature_names[i] for i in top_word_indices]
    top_word_probs = [topic_word_distribution[i] for i in top_word_indices]
    return top_words

### target 단어 추출

In [8]:
# target 데이터 불러오기
art_txt = 'synopsis_art.txt'
gen_txt = 'synopsis_gen.txt'
# 품사에 따라 바꾸기
output_art_txt = 'synopsis_art_nouns.txt'
output_gen_txt = 'synopsis_gen_nouns.txt'

'''
save_tokens(art_txt, output_art_txt)
save_tokens(gen_txt, output_gen_txt)
'''

art=read_tokens(output_art_txt)
gen=read_tokens(output_gen_txt)

In [9]:
# 벡터화, 불용어 제거
vec_type = 'tfidf'

# TF-IDF
if vec_type=='tfidf':
    vectorizer = TfidfVectorizer(stop_words=stopwords)
# DTM
elif vec_type=='dtm':
    vectorizer = CountVectorizer(stop_words=stopwords)
    
X = vectorizer.fit_transform([art,gen])

In [10]:
m1 = X[0].tocoo()   # sparse matrix를 가져오기
m2 = X[1].tocoo()   # sparse matrix를 가져오기

w1 = [[i, j] for i, j in zip(m1.col, m1.data)]
w2 = [[i, j] for i, j in zip(m2.col, m2.data)]

w1.sort(key=lambda x: x[1], reverse=True)   #art를 구성하는 단어들을 높은 순으로 정렬
w2.sort(key=lambda x: x[1], reverse=True)   #gen을 구성하는 단어들을 높은 순으로 정렬 

w1_, w2_ = [], []
print('예술영화를 대표하는 단어들:')
for i in range(100):
    word = vectorizer.get_feature_names()[w1[i][0]]
    print(word, end=', ')
    w1_.append(word)

print('\n')
    
print('일반영화를 대표하는 단어들:')
for i in range(100):
    word = vectorizer.get_feature_names()[w2[i][0]]
    print(word, end=', ')
    w2_.append(word)

예술영화를 대표하는 단어들:
사랑, 사람, 친구, 남자, 가족, 

/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


이야기, 마을, 사건, 마음, 세상, 아버지, 아이, 엄마, 모든, 여자, 대한, 서로, 다시, 시간, 아들, 소녀, 아내, 다른, 사이, 영화제, 세계, 사실, 하나, 남편, 감독, 여행, 인생, 발견, 모두, 순간, 우리, 가장, 마지막, 생활, 아빠, 모습, 죽음, 기억, 비밀, 학교, 음악, 한편, 소년, 생각, 도시, 명의, 사고, 결혼, 전쟁, 위기, 최고, 이자, 과거, 일상, 경찰, 상황, 간다, 미국, 결심, 운명, 현실, 관계, 지금, 단편, 여인, 하루, 이름, 이후, 준비, 인간, 감정, 만난, 국제, 처음, 충격, 살인, 누구, 동안, 존재, 그린, 어머니, 연인, 계속, 동생, 작품, 청년, 한국, 가지, 상처, 할머니, 목숨, 이상, 희망, 계획, 매력, 

일반영화를 대표하는 단어들:
영화제, 사람, 국제, 친구, 사랑, 남자, 이야기, 대한, 서울, 여자, 사건, 남편, 아이, 가족, 아버지, 다른, 마을, 시간, 엄마, 아들, 모든, 단편, 마음, 사실, 다시, 세계, 모습, 작품, 생각, 서로, 세상, 발견, 감독, 아내, 관계, 소녀, 사이, 하나, 우리, 애니메이션, 여성, 죽음, 인간, 생활, 한편, 결혼, 상황, 모두, 기억, 명의, 소년, 여행, 가장, 간다, 순간, 도시, 비밀, 학교, 과거, 가지, 이자, 경찰, 마지막, 미국, 동안, 전쟁, 주인공, 대해, 존재, 현실, 연출, 사고, 살인, 일상, 어머니, 계속, 사회, 인생, 다큐멘터리, 부문, 섹스, 최고, 바로, 동생, 의도, 하루, 위기, 계획, 정체, 한국, 이후, 조직, 회사, 보고, 부산, 아빠, 부부, 일본, 문제, 처음, 

> Q. 왜 두 문서 사이에 중복값이 많을까?(하다, 되다, 그녀, 자신, 위해 등)  
> A. 둘 다 영화를 다루고 있으므로 공통된 어휘가 많이 사용됨. 또한 현재 문서가 2개만 존재하므로 단어 간의 IDF 값 차이가 적어져 TF 값이 TF-IDF 값에 영향을 끼치는 정도가 커짐

#### TF-IDF/DTM 그대로 활용

In [11]:
# 중복 제거
n = 15

# w1에만 있고 w2에는 없는, 예술영화를 잘 대표하는 단어를 15개 추출한다.
target_art, target_gen = [], []
for i in range(100):
    if (w1_[i] not in w2_) and (w1_[i] in model.wv): target_art.append(w1_[i])
    if len(target_art) == n: break 

# w2에만 있고 w1에는 없는, 일반영화를 잘 대표하는 단어를 15개 추출한다.
for i in range(100):
    if (w2_[i] not in w1_) and (w2_[i] in model.wv): target_gen.append(w2_[i])
    if len(target_gen) == n: break

In [12]:
print('예술영화를 대표하는 단어들:')
for word in target_art:
    print(word, end=', ')

print('\n')
    
print('일반영화를 대표하는 단어들:')
for word in target_gen:
    print(word, end=', ')

예술영화를 대표하는 단어들:
음악, 결심, 운명, 지금, 여인, 이름, 준비, 감정, 만난, 충격, 누구, 그린, 연인, 청년, 상처, 

일반영화를 대표하는 단어들:
서울, 애니메이션, 여성, 주인공, 대해, 연출, 사회, 다큐멘터리, 부문, 섹스, 바로, 의도, 정체, 조직, 회사, 

#### LDA 적용

In [13]:
X.shape

(2, 41088)

In [14]:
# LDA
from sklearn.decomposition import LatentDirichletAllocation

lda_model = LatentDirichletAllocation(n_components=2, 
                                      learning_method='online', 
                                      random_state=777, 
                                      max_iter=10)

lda_model.fit_transform(X)

array([[0.00558322, 0.00558358, 0.00558402, 0.97766499, 0.00558419],
       [0.00556495, 0.00556564, 0.00556601, 0.97773719, 0.00556621]])

In [15]:
# LDA의 결과 토픽과 각 단어의 비중을 출력
n_top_words = 30
tf_feature_names = vectorizer.get_feature_names_out()

print("Topics in LDA model:")
print_top_words(lda_model, tf_feature_names, n_top_words)

print("Document-Topic Distributions:")
documents = [art, gen]
max_topic_idxs = print_max_topic_for_documents(lda_model, documents)

Topics in LDA model:
Topic #0:
클린콘텐츠 강혁 한펑 키프로스 에스포 에리얼 혹세 토사 정작 갈수록 메스 들락거렸다 유동 대리시 리카르도 박사 테리블 별미 조작 깅코 아널드 마크 각성 신동 태세 쿠오 종강 세익스피어 러브스토리 라이트
Topic #1:
충언 불충분 판이 카위 복장 호전 메이든 지건뎃 주스 리셋 골수 클리닝 제주시 실질 흥수 조준 커즌스 어인 코브햄 훗카이도 무마 노동운동가 후지야 대학교수 완만 고고학자 저능 벼랑 캠벨 로소
Topic #2:
베라크루즈 김현주 정경 결승전 취소 영준 은행 국가안보국 세실리아 학도 무르만스크 스트라이커 후퇴 나카마 대머리 이노 점정 스페어 펠레우스 홀딩 살갑 서고 까뮈 최민석 승과 모우 아웃팅 훔볼트 보루 철저
Topic #3:
사람 사랑 영화제 친구 남자 이야기 국제 가족 대한 아이 여자 다른 아버지 세상 마을 소녀 남편 마음 단편 모든 시간 사실 결혼 사건 간다 다시 아내 순간 감독 서로
Topic #4:
단도 퀴퍼스 주물 종군위안부 허일 모진 미소노 용맹 고요 인승 마리아 브리즈번 저수지 바오산 레오파드 델우드 사냥터 에스콘디도 유혹 관태 고민 맥퍼슨 빈자 음경 주영철 부처 너구리 미남자 라자스탄 고려

Document-Topic Distributions:
Document #0:
Max Probability Topic: 3, Probability: 0.9777

Document #1:
Max Probability Topic: 3, Probability: 0.9777



In [ ]:
target_art = get_topic_words(lda_model, max_topic_idxs[0], tf_feature_names, 15)
target_gen = get_topic_words(lda_model, max_topic_idxs[1], tf_feature_names, 15)

### attribute 단어 추출 

In [26]:
#파일 불러와서 TF-IDF 생성
genre_txt = ['synopsis_SF.txt', 'synopsis_family.txt', 'synopsis_show.txt', 'synopsis_horror.txt', 'synopsis_etc.txt', 
             'synopsis_documentary.txt', 'synopsis_drama.txt', 'synopsis_romance.txt', 'synopsis_musical.txt', 
             'synopsis_mystery.txt', 'synopsis_crime.txt', 'synopsis_historical.txt', 'synopsis_western.txt', 
             'synopsis_adult.txt', 'synopsis_thriller.txt', 'synopsis_animation.txt', 'synopsis_action.txt', 
             'synopsis_adventure.txt', 'synopsis_war.txt', 'synopsis_comedy.txt', 'synopsis_fantasy.txt']
genre_name = ['SF', '가족', '공연', '공포(호러)', '기타', '다큐멘터리', '드라마', '멜로로맨스', '뮤지컬', '미스터리', '범죄', '사극', '서부극(웨스턴)',
         '성인물(에로)', '스릴러', '애니메이션', '액션', '어드벤처', '전쟁', '코미디', '판타지']
# 품사에 따라 바꾸기
output_genre_txt = [txt.strip('.txt')+'_nouns.txt' for txt in genre_txt]

genre = []
for file_name, output_file_name in zip(genre_txt, output_genre_txt):
    #save_tokens(file_name, output_file_name)
    genre.append(read_tokens(output_file_name))

In [27]:
# 벡터화, 불용어 제거
# TF-IDF
if vec_type=='tfidf':
    vectorizer = TfidfVectorizer(stop_words=stopwords)
# DTM
elif vec_type=='dtm':
    vectorizer = CountVectorizer(stop_words=stopwords)
X = vectorizer.fit_transform(genre)

#### TF-IDF/DTM 그대로 사용

In [28]:
m = [X[i].tocoo() for i in range(X.shape[0])]

w = [[[i, j] for i, j in zip(mm.col, mm.data)] for mm in m]

for i in range(len(w)):
    w[i].sort(key=lambda x: x[1], reverse=True)

attributes = []
for i in range(len(w)):
    print(genre_name[i], end=': ')
    attr = []
    j = 0
    while (len(attr) < 15):
        if vectorizer.get_feature_names()[w[i][j][0]] in model.wv:
            attr.append(vectorizer.get_feature_names()[w[i][j][0]])
            print(vectorizer.get_feature_names()[w[i][j][0]], end=', ')
        j += 1
    attributes.append(attr)
    print()

SF: 지구, 사람, 

/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


인류, 인간, 미래, 우주, 로봇, 세계, 모든, 박사, 우주선, 외계, 존재, 세상, 발견, 
가족: 엄마, 아빠, 가족, 영화제, 친구, 아주르, 아버지, 아들, 마을, 국제, 낙타, 할머니, 씨제이, 동구, 사랑, 
공연: 오페라, 사랑, 토스카, 실황, 올레, 카바, 공연, 오텔로, 리골레토, 백작, 프레, 베르디, 카르피, 비바, 왕자, 
공포(호러): 사람, 친구, 사건, 공포, 발견, 죽음, 마을, 가족, 악령, 남자, 좀비, 사실, 소녀, 하나, 살인, 
기타: 영화제, 국제, 서울, 단편, 사람, 이야기, 남자, 사랑, 뉴미디어, 페스티벌, 여자, 대한, 독립, 친구, 작품, 
다큐멘터리: 영화제, 다큐, 국제, 다큐멘터리, 사람, 이야기, 대한, 감독, 서울, 우리, 세계, 여성, 가족, 한국, 작품, 
드라마: 영화제, 사람, 사랑, 국제, 남자, 친구, 이야기, 엄마, 여자, 아버지, 가족, 단편, 서울, 대한, 아들, 
멜로로맨스: 사랑, 남편, 남자, 여자, 사람, 친구, 섹스, 마음, 결혼, 서로, 아내, 관계, 부부, 엄마, 아버지, 
뮤지컬: 뮤지컬, 사랑, 에스메랄다, 음악, 충무로, 모차르트, 토스카, 니웨, 카바, 영화제, 바흐, 페뷔스, 프롤, 모도, 카르피, 
미스터리: 사건, 사람, 발견, 사고, 진실, 죽음, 기억, 살인, 친구, 아내, 남자, 아이, 민혁, 사실, 의문, 
범죄: 사건, 경찰, 범죄, 조직, 살인, 사람, 마약, 형사, 남자, 모든, 살해, 수사, 발견, 한길수, 범인, 
사극: 조선, 신기전, 사랑, 아가멤논, 황제, 루안, 최고, 운명, 사람, 하선, 전쟁, 윤서, 트로이, 세자, 허균, 
서부극(웨스턴): 서부, 보안관, 벌린, 카우보이, 그레이프바인, 헨리, 마을, 개릿, 아이, 무법자, 프린트, 마적, 태구, 현상금, 분노, 
성인물(에로): 남편, 마사지, 섹스, 관계, 정사, 남자, 여자, 유부녀, 마음, 사랑, 에피소드, 그린, 아내, 다시, 자위, 
스릴러: 사건, 사람, 살인, 남자

#### LDA 적용

In [29]:
# LDA
lda_model = LatentDirichletAllocation(n_components=50, 
                                      learning_method='online', 
                                      random_state=777, 
                                      max_iter=10,
                                     doc_topic_prior=0.1)
# 주제 간 중복 줄이기 위해 topic_word_prior 조정 가능
# 문서 간 주제 중복 줄이기 위해 doc_topic_prior 조정 가능

lda_model.fit_transform(X)

/opt/conda/lib/python3.9/site-packages/sklearn/decomposition/_lda.py:881: RuntimeWarning: overflow encountered in exp
  return np.exp(-1.0 * perword_bound)


array([[0.00236818, 0.00236818, 0.00236818, ..., 0.00236819, 0.00236819,
        0.00236819],
       [0.00287849, 0.00287847, 0.00287847, ..., 0.00287849, 0.00287847,
        0.00287848],
       [0.00316709, 0.00316708, 0.00316709, ..., 0.00316712, 0.00316709,
        0.00316709],
       ...,
       [0.00282587, 0.00282586, 0.00282585, ..., 0.00282588, 0.00282589,
        0.00282587],
       [0.00174136, 0.00174136, 0.00174136, ..., 0.00174136, 0.00174136,
        0.00174136],
       [0.00248986, 0.00248985, 0.00248986, ..., 0.00248987, 0.00248986,
        0.00248986]])

In [30]:
# LDA의 결과 토픽과 각 단어의 비중을 출력
n_top_words = 15
tf_feature_names = vectorizer.get_feature_names_out()

print("Topics in LDA model:")
print_top_words(lda_model, tf_feature_names, n_top_words)

print("Max Probability Topic for Each Document:")
documents = genre
max_topic_idxs = print_max_topic_for_documents(lda_model, documents)

Topics in LDA model:
Topic #0:
케닝턴 강태 프란체스코 켄톤 얏타킹 야크 해갈 큰손 갈리아 저금 맥거핀 운수 담시 뒤안길 류승완
Topic #1:
지적 범생 퀜틴 쭈욱 발발 화련 프랭코 리벨 전수 재결합 뚝방파 개척지 추쉐이 자금지원 순찰차
Topic #2:
몽롱 회문 골치 음악회 조원 아네믹 연차 블랙리스트 강매 태평 롱쇼 생활사 폭포 광위 피앙세
Topic #3:
윌슨 스나이더 닝징 손혜윤 커즌스 전제정치 업친 웨폰 하일 박진 인공위성 감서 레위기 캬바레 아킴
Topic #4:
고래사냥 정권 워크맨 우주망원경 컵케이크 대한문 동학혁명 슬하 얌체 혼성 영혼 메린 노닥 딱지왕 김태우
Topic #5:
액셀 사다코 검거 왈도 통칭 붐바 그린북 여름날 닥터 성인영화 프린세스 고서점 택했던 기관총 당진시
Topic #6:
먼지 요코하마 수욱 자만 신장 엠마뉴엘 섹쉬궁 조혁 스테롤 카시라 숙련 뉴스메이커 슈퍼컴퓨터 냄비 냠걀사원
Topic #7:
베렛 캐리 피오 자살 총사령관 골프장 츄츄 정보기관 병태 인지도 멸치 크리슈나 관성 이윤호 할아범
Topic #8:
열망 호창 횡설수설 수혈 연구실 서브프라임 계심 직상 아마게돈 플라멩코 프레임 실종자 러스트 용케 인영
Topic #9:
영화제 국제 사람 단편 서울 사랑 가족 친구 남자 소녀 아이 대한 이야기 적의 디스플레이
Topic #10:
미술감독 새봄 맥켈웨이 치약 별탈 허드슨만 크림반도 잠수 타이스 선량 최정 뻔뻔 애하 하천 저격총
Topic #11:
해걸 신성모독 백도 격없 파타콘 제드 심청 계단 산아 빛낼 치환 색쉬함 미츠키 가부 사빈
Topic #12:
상륙작전 츠쿠시 상대편 길거리 홍역 리페 사신 종말 마스 클로저 메츠 쏙쏙 스파르티 렉시스 막스
Topic #13:
아모레 경제위기 도둑 경시청 김정학 랜드마크 장착 조폭 모노크롬 랩퍼 강시 통강 상대자 텡보체 결찰
Topic #14:
동국대학교 텟페 종진 천방지축 간극 마크 위업 룰로 연제욱 이슬람 키파 연예계 모음곡 워터마크 미켈란젤로

In [ ]:
attributes=[]
for i in range(len(genre_)):
    attributes.append(get_topic_words(lda_model, max_topic_idxs[i], tf_feature_names, 15))

## STEP 4. WEAT score 계산과 시각화

In [23]:
from numpy import dot
from numpy.linalg import norm
# WEAT score function
def cos_sim(i, j):
    return dot(i, j.T)/(norm(i)*norm(j))
def s(w, A, B):
    c_a = cos_sim(w, A)
    c_b = cos_sim(w, B)
    mean_A = np.mean(c_a, axis=-1)
    mean_B = np.mean(c_b, axis=-1)
    return mean_A - mean_B #, c_a, c_b
def weat_score(X, Y, A, B):    
    s_X = s(X, A, B)
    s_Y = s(Y, A, B)

    mean_X = np.mean(s_X)
    mean_Y = np.mean(s_Y)
    
    std_dev = np.std(np.concatenate([s_X, s_Y], axis=0))
    
    return  (mean_X-mean_Y)/std_dev

In [24]:
# embedding model과 단어 셋으로 WEAT score 구해보기
matrix = [[0 for _ in range(len(genre_name))] for _ in range(len(genre_name))]
X = np.array([model.wv[word] for word in target_art])
Y = np.array([model.wv[word] for word in target_gen])

for i in range(len(genre_name)-1):
    for j in range(i+1, len(genre_name)):
        A = np.array([model.wv[word] for word in attributes[i]])
        B = np.array([model.wv[word] for word in attributes[j]])
        matrix[i][j] = weat_score(X, Y, A, B)
        
for i in range(len(genre_name)-1):
    for j in range(i+1, len(genre_name)):
        print(genre_name[i], genre_name[j],matrix[i][j])

SF 가족 -0.62751734
SF 공연 -0.31261918
SF 공포(호러) -0.7071042
SF 기타 0.053081196
SF 다큐멘터리 0.3014637
SF 드라마 -0.5334929
SF 멜로로맨스 -0.8641359
SF 뮤지컬 0.11361934
SF 미스터리 -0.6645717
SF 범죄 0.02021459
SF 사극 -0.65355295
SF 서부극(웨스턴) -0.38777822
SF 성인물(에로) -0.6842467
SF 스릴러 -0.54667616
SF 애니메이션 0.16000181
SF 액션 -0.12658557
SF 어드벤처 -1.0643228
SF 전쟁 0.38560516
SF 코미디 -0.7912917
SF 판타지 -0.44812104
가족 공연 0.43599573
가족 공포(호러) -0.018791296
가족 기타 0.7808947
가족 다큐멘터리 0.8369918
가족 드라마 0.21816558
가족 멜로로맨스 -0.76249176
가족 뮤지컬 0.8899251
가족 미스터리 -0.12492989
가족 범죄 0.5542809
가족 사극 0.28596336
가족 서부극(웨스턴) 0.7796064
가족 성인물(에로) -0.5068402
가족 스릴러 -0.024650745
가족 애니메이션 0.918894
가족 액션 0.5403716
가족 어드벤처 -0.32417694
가족 전쟁 0.90272003
가족 코미디 -0.25929192
가족 판타지 0.5170888
공연 공포(호러) -0.23789923
공연 기타 0.69263273
공연 다큐멘터리 0.7110423
공연 드라마 -0.3330407
공연 멜로로맨스 -0.7960861
공연 뮤지컬 1.0654726
공연 미스터리 -0.2888558
공연 범죄 0.27542812
공연 사극 -0.22270049
공연 서부극(웨스턴) 0.056426007
공연 성인물(에로) -0.65336853
공연 스릴러 -0.22501819
공연 애니메이션 0.78756833
공연 액션 0.2112

In [ ]:
# 시각화
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

def plot_heatmap(matrix, genre_name, title, save=True):
    np.random.seed(0)
    # 한글 지원 폰트
    sns.set(font='NanumGothic')

    # 마이너스 부호 
    plt.rcParams['axes.unicode_minus'] = False

    fig, ax = plt.subplots(figsize=(15,13))
    sns.heatmap(matrix, xticklabels=genre_name, yticklabels=genre_name, annot=True, cmap='RdYlGn_r', ax=ax)

    ax.set_title(title, fontsize=20, pad=20)
    if save:
        plt.savefig('heatmap_'+title+'.png', bbox_inches='tight')

    plt.show()
    
plot_heatmap(matrix, genre_name, 'LDA', False)